In [ ]:
# Only execute on Colab
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Install packages
    %pip install circuitsvis
    %pip install transformer_lens

    # Code to make sure output widgets display
    from google.colab import output

    output.enable_custom_widget_manager()

    !wget -q https://github.com/EffiSciencesResearch/ML4G-2.0/archive/refs/heads/master.zip
    !unzip -o /content/master.zip 'ML4G-2.0-master/workshops/transformer_interp/Circuits/*'
    !mv --no-clobber ML4G-2.0-master/workshops/transformer_interp/Circuits/* .
    !rm -r ML4G-2.0-master

    print("Imports & installations complete!")

else:
    from IPython import get_ipython

    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G-2.0/blob/master/workshops/transformer_interp/Circuits/easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
## Introduction

Thanks to Callum McDougall for creating mauch of this notebooks content!

This exercise should get you familiar with concepts in circuits interpretability in transformers.

## Content & Learning Objectives

#### 1️⃣ Find Inductive Heads via attention patterns

You'll plot attnetion patterns to check out theory of Inductive Heads, and see if you can find them in the model you're working with.
> ##### Learning objectives
> 
> - Use `circuitsvis` to visualise attention heads
> - Understand what the theory of inductive heads predicts about attention patterns
> - Use attention patterns to identify inductive heads
> - Automate this process to find inductive heads in a larger model

#### 2️⃣ Logit Attribution

Here, you'll learn how to use TransfomerLens to implement LogitLens, a tool for attributing logit values to specific components of the model. You'll also learn how to use this tool to identify basic attention heads, that are imortant for Induction tasks
> ##### Learning objectives
> - Perform direct logit attribution to figure out which heads are writing to the residual stream in a significant way
> - Crosscheck your earlier restuls with the results




In [ ]:
import os

os.environ["ACCELERATE_DISABLE_RICH"] = "1"

import torch as t
from torch import Tensor
import numpy as np
from tqdm.notebook import tqdm
from jaxtyping import Float, Int, Bool
import circuitsvis as cv
from transformer_lens import utils, HookedTransformer, ActivationCache
from transformer_lens import HookedTransformerConfig
from huggingface_hub import hf_hub_download
from tests import test_logit_attribution, test_current_attn_detector, test_average_over_condition, test_first_attn_detector, test_induction_attn_detector, test_prev_attn_detector
t.set_grad_enabled(False)
device = t.device("cuda") if t.cuda.is_available() else t.device("cpu")

## Loading and Running Models

We will be using two models today:

GPT-2 small, a smaller version of the GPT-2 model, with 12 layers and 80 million parameters.

A two-layer attention only transfomer model.

This toy model is a 2L attention-only transformer trained specifically for today. Some changes to make them easier to interpret:
- It has only attention blocks.
- The positional embeddings are only added to the residual stream before each key and query vector in the attention layers as opposed to the token embeddings - i.e. we compute queries as `Q = (resid + pos_embed) @ W_Q + b_Q` and same for keys, but values as `V = resid @ W_V + b_V`. This means that **the residual stream can't directly encode positional information**.
    - This turns out to make it *way* easier for induction heads to form, it happens 2-3x times earlier - [see the comparison of two training runs](https://wandb.ai/mechanistic-interpretability/attn-only/reports/loss_ewma-22-08-24-11-08-83---VmlldzoyNTI0MDMz?accessToken=8ap8ir6y072uqa4f9uinotdtrwmoa8d8k2je4ec0lyasf1jcm3mtdh37ouijgdbm) here. (The bump in each curve is the formation of induction heads.)
    - The argument that does this below is `positional_embedding_type="shortformer"`.
- It has no MLP layers, no LayerNorms, and no biases.
- There are separate embed and unembed matrices (i.e. the weights are not tied).


In [ ]:
gpt2_small: HookedTransformer = HookedTransformer.from_pretrained("gpt2-small")
gpt2_small.set_use_attn_result(True)

cfg = HookedTransformerConfig(
    d_model=768,
    d_head=64,
    n_heads=12,
    n_layers=2,
    n_ctx=2048,
    d_vocab=50278,
    attention_dir="causal",
    attn_only=True,  # defaults to False
    tokenizer_name="EleutherAI/gpt-neox-20b",
    seed=398,
    use_attn_result=True,
    normalization_type=None,  # defaults to "LN", i.e. layernorm with weights & biases
    positional_embedding_type="shortformer",
)

REPO_ID = "callummcdougall/attn_only_2L_half"
FILENAME = "attn_only_2L_half.pth"

weights_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
toy_transformer: HookedTransformer = HookedTransformer(cfg)

toy_transformer.load_state_dict(t.load(weights_path, map_location=device))



## Caching all Activations

The first basic operation when doing mechanistic interpretability is to break open the black box of the model and look at all of the internal activations of a model. This can be done with `logits, cache = model.run_with_cache(tokens)`. Let's try this out, on the first sentence from the GPT-2 paper.

<details>
<summary>Aside - a note on <code>remove_batch_dim</code></summary>

Every activation inside the model begins with a batch dimension. Here, because we only entered a single batch dimension, that dimension is always length 1 and kinda annoying, so passing in the `remove_batch_dim=True` keyword removes it. 

`gpt2_cache_no_batch_dim = gpt2_cache.remove_batch_dim()` would have achieved the same effect.
</details>

In [ ]:
text = "Harry James Potter Evans-Verres went to ML4good, to learn about transformers, that is why Harry James Potter Evans-Verres did this notebook in ML4good."

gpt_tokens = gpt2_small.to_tokens(text)
toymodel_tokens = toy_transformer.to_tokens(text)

gpt_logits, gpt_cache = gpt2_small.run_with_cache(gpt_tokens, remove_batch_dim=True)
toymodel_logits, toymodel_cache = toy_transformer.run_with_cache(toymodel_tokens, remove_batch_dim=True)

If you inspect the `gpt_cache` object, you should see that it contains a very large number of keys, each one corresponding to a different activation in the model. You can access the keys by indexing the cache directly, or by a more convenient indexing shorthand. For instance, the code:


In [ ]:
gpt_attn_patterns_layer_0 = gpt_cache["pattern", 0]
toymodel_attn_patterns_layer_0 = toymodel_cache["pattern", 0]

returns the same thing as:

In [ ]:
gpt_attn_patterns_layer_0_copy = gpt_cache["blocks.0.attn.hook_pattern"]

t.testing.assert_close(gpt_attn_patterns_layer_0, gpt_attn_patterns_layer_0_copy)


<details>
<summary>Aside: <code>utils.get_act_name</code></summary>

The reason these are the same is that, under the hood, the first example actually indexes by `utils.get_act_name("pattern", 0)`, which evaluates to `"blocks.0.attn.hook_pattern"`.

In general, `utils.get_act_name` is a useful function for getting the full name of an activation, given its short name and layer number.

You can use the diagram from the **Transformer Architecture** section to help you find activation names.
</details>




## Visualising Attention Heads

A key insight from the Mathematical Frameworks paper is that we should focus on interpreting the parts of the model that are intrinsically interpretable - the input tokens, the output logits and the attention patterns. Everything else (the residual stream, keys, queries, values, etc) are compressed intermediate states when calculating meaningful things. So a natural place to start is classifying heads by their attention patterns on various texts.

When doing interpretability, it's always good to begin by visualising your data, rather than taking summary statistics. Summary statistics can be super misleading! But now that we have visualised the attention patterns, we can create some basic summary statistics and use our visualisations to validate them! (Accordingly, being good at web dev/data visualisation is a surprisingly useful skillset! Neural networks are very high-dimensional object.)

Let's visualize the attention pattern of all the heads in layer 0, using [Alan Cooney's CircuitsVis library](https://github.com/alan-cooney/CircuitsVis) (based on Anthropic's PySvelte library). If you did the previous set of exercises, you'll have seen this library before.

We will use the function `cv.attention.attention_patterns`, which takes the following arguments:

* `attention`: Attention head activations. 
    * This should be a tensor of shape `[nhead, seq_dest, seq_src]`, i.e. the `[i, :, :]`th element is the grid of attention patterns (probabilities) for the `i`th attention head.
    * We get this by indexing our `gpt2_cache` object.
* `tokens`: List of tokens (e.g. `["A", "person"]`). 
    * Sequence length must match that inferred from `attention`.
    * This is used to label the grid.
    * We get this by using the `gpt2_small.to_str_tokens` method.
* `attention_head_names`: Optional list of names for the heads.

There are also other circuitsvis functions, e.g. `cv.attention.attention_pattern` (for just a single head) or `cv.attention.attention_heads` (which has the same syntax and but presents the information in a different form).

<details>
<summary>Help - my <code>attention_heads</code> plots are behaving weirdly (e.g. they continually shrink after I plot them).</summary>

This seems to be a bug in `circuitsvis` - on VSCode, the attention head plots continually shrink in size.

Until this is fixed, one way to get around it is to open the plots in your browser. You can do this inline with the `webbrowser` library:

```python
attn_heads = cv.attention.attention_heads(
    tokens=gpt2_str_tokens, 
    attention=attention_pattern,
    attention_head_names=[f"L0H{i}" for i in range(12)],
)

path = "attn_heads.html"

with open(path, "w") as f:
    f.write(str(attn_heads))

webbrowser.open(path)
```

To check exactly where this is getting saved, you can print your current working directory with `os.getcwd()`.
</details>

This visualization is interactive! Try hovering over a token or head, and click to lock. The grid on the top left and for each head is the attention pattern as a destination position by source position grid. It's lower triangular because GPT-2 has **causal attention**, attention can only look backwards, so information can only move forwards in the network.


In [ ]:
print(type(gpt_cache))
attention_pattern = gpt_cache["pattern", 10]
print(attention_pattern.shape)
str_toknes = gpt2_small.to_str_tokens(text)

print("Layer 0 Head Attention Patterns:")
display(
    cv.attention.attention_patterns(
        tokens=str_toknes,
        attention=attention_pattern
    )
)


Hover over heads to see the attention patterns; click on a head to lock it. Hover over each token to see which other tokens it attends to (or which other tokens attend to it - you can see this by changing the dropdown to `Destination <- Source`).

<details>
<summary>Other circuitsvis functions - neuron activations</summary>

The `circuitsvis` library also has a number of cool visualisations for **neuron activations**. Here are some more of them (you don't have to understand them all now, but you can come back to them later).

The function below visualises neuron activations. The example shows just one sequence, but it can also show multiple sequences (if `tokens` is a list of lists of strings, and `activations` is a list of tensors).

```python
neuron_activations_for_all_layers = t.stack([
    gpt2_cache["post", layer] for layer in range(gpt2_small.cfg.n_layers)
], dim=1)
# shape = (seq_pos, layers, neurons)

cv.activations.text_neuron_activations(
    tokens=gpt2_str_tokens,
    activations=neuron_activations_for_all_layers
)
```

The next function shows which words each of the neurons activates most / least on (note that it requires some weird indexing to work correctly).

```python
neuron_activations_for_all_layers_rearranged = utils.to_numpy(einops.rearrange(neuron_activations_for_all_layers, "seq layers neurons -> 1 layers seq neurons"))

cv.topk_tokens.topk_tokens(
    # Some weird indexing required here ¯\_(ツ)_/¯
    tokens=[gpt2_str_tokens], 
    activations=neuron_activations_for_all_layers_rearranged,
    max_k=7, 
    first_dimension_name="Layer", 
    third_dimension_name="Neuron",
    first_dimension_labels=list(range(12))
)
```
</details>

# Finding induction heads



             
Use the [diagram at this link](https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/small-merm.svg) to remind yourself of the relevant hook names.


### Exercise - visualise attention patterns

```yaml
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to ~10 minutes on this exercise.

It's important to be comfortable using circuitsvis, and the cache object.
```

*This exercise should be very quick - you can reuse code from the previous section. You should look at the solution if you're still stuck after 5-10 minutes.*

Visualise the attention patterns for both layers of your model, on the following prompt:


In [ ]:
# YOUR CODE HERE - visualize attention

<details>
<summary>Solution </summary>

We visualise attention patterns with the following code:

```python
str_tokens = model.to_str_tokens(text)
for layer in range(model.cfg.n_layers):
    attention_pattern = cache["pattern", layer]
    display(cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern))
```

We notice that there are three basic patterns which repeat quite frequently:

* `prev_token_heads`, which attend mainly to the previous token (e.g. head `0.7`)
* `current_token_heads`, which attend mainly to the current token (e.g. head `1.6`)
* `first_token_heads`, which attend mainly to the first token (e.g. heads `0.3` or `1.4`, although these are a bit less clear-cut than the other two)

The `prev_token_heads` and `current_token_heads` are perhaps unsurprising, because words that are close together in a sequence probably have a lot more mutual information (i.e. we could get quite far using bigram or trigram prediction). 

The `first_token_heads` are a bit more surprising. The basic intuition here is that the first token in a sequence is often used as a resting or null position for heads that only sometimes activate (since our attention probabilities always have to add up to 1).
</details>


Now that we've observed our three basic attention patterns, it's time to make detectors for those patterns!


### Exercise - write your own detectors

```yaml
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You shouldn't spend more than 15-20 minutes on these exercises.

These exercises shouldn't be too challenging, if you understand attention patterns. Use the hint if stuck on things like how to correctly index your tensors, or how to access the activation patterns from the cache.
```

You should fill in the functions below, which act as detectors for particular types of heads. Validate your detectors by comparing these results to the visual attention patterns above - summary statistics on their own can be dodgy, but are much more reliable if you can validate it by directly playing with the data.

Tasks like this are useful, because we need to be able to take our observations / intuitions about what a model is doing, and translate these into quantitative measures. As the exercises proceed, we'll be creating some much more interesting tools and detectors!

Note - there's no objectively correct answer for which heads are doing which tasks, and which detectors can spot them. You should just try and come up with something plausible-seeming, which identifies the kind of behaviour you're looking for.



In [ ]:

def average_over_condition(tensor, condition):
    I, J, K = tensor.shape
    return [
        sum(tensor[i, j, k] for j in range(J) for k in range(K) if condition(j, k))
        / sum(condition(j, k) for j in range(J) for k in range(K))
        for i in range(I)
    ]


def over_threshhold_attn(cache, condition, threshhold=0.5, sorce="pattern"):
    return_values = []

    for layer, pattern in enumerate( cache.stack_activation("pattern")):
        scores = average_over_condition(pattern, condition)
        indices = [i for i, s in enumerate(scores) if s > threshhold]
        for i in indices:
            return_values.append(f"L{layer+1}H{i}")
    return return_values


def current_attn_detector(cache: ActivationCache, threshhold = .3) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be current-token heads
    """

    pass


def prev_attn_detector(cache: ActivationCache,threshhold =.3 ) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be prev-token heads
    """

    pass


def first_attn_detector(cache: ActivationCache, threshhold = .3) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be first-token heads
    """

    pass

def find_repeating_rows(tensor):
    """
    Finds repeating rows (vectors) in a 2D torch tensor.

    Args:
    tensor (torch.Tensor): A 2D torch tensor.

    Returns:
    dict: A dictionary where keys are the indices of repeating rows,
          and values are the indices where those rows last occurred.
    """
    last_occurrence = {}
    repeats = {}

    for pos, token in enumerate(tensor[0]):
        id  = token.item()

        if id in last_occurrence:
            repeats[pos] = last_occurrence[id]
        last_occurrence[id] = pos
    
    return repeats

def induction_attn_detector(cache: ActivationCache, tokens, off_by_one = True, threshhold = .3) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be induction heads

    Remember - the tokens used to generate rep_cache are (bos_token, *rand_tokens, *rand_tokens)
    """
    repeat_dict = find_repeating_rows(t.tensor(tokens))

    pass


test_current_attn_detector(current_attn_detector, toy_transformer)
test_first_attn_detector(first_attn_detector, toy_transformer)
test_prev_attn_detector(prev_attn_detector, toy_transformer)
test_induction_attn_detector(induction_attn_detector, toy_transformer)




<details>

<summary>Solution</summary>


```python
def average_over_condition(tensor, condition):
    I, J, K = tensor.shape
    return [
        sum(tensor[i, j, k] for j in range(J) for k in range(K) if condition(j, k))
        / sum(condition(j, k) for j in range(J) for k in range(K))
        for i in range(I)
    ]


def over_threshhold_attn(cache, condition, threshhold=0.5, sorce="pattern"):
    return_values = []

    for layer, pattern in enumerate( cache.stack_activation("pattern")):
        scores = average_over_condition(pattern, condition)
        indices = [i for i, s in enumerate(scores) if s > threshhold]
        for i in indices:
            return_values.append(f"L{layer+1}H{i}")
    return return_values


def current_attn_detector(cache: ActivationCache, threshhold = .3) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be current-token heads
    """

    def cond(i, j):
        return i == j

    return over_threshhold_attn(cache, cond, threshhold=threshhold)


def prev_attn_detector(cache: ActivationCache,threshhold =.3 ) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be prev-token heads
    """

    def cond(i, j):
        return i - j == 1

    return over_threshhold_attn(cache, cond, threshhold=threshhold)


def first_attn_detector(cache: ActivationCache, threshhold = .3) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be first-token heads
    """

    def cond(i, j):
        return j == 0

    return over_threshhold_attn(cache, cond, threshhold=threshhold)

def find_repeating_rows(tensor):
    """
    Finds repeating rows (vectors) in a 2D torch tensor.

    Args:
    tensor (torch.Tensor): A 2D torch tensor.

    Returns:
    dict: A dictionary where keys are the indices of repeating rows,
          and values are the indices where those rows last occurred.
    """
    last_occurrence = {}
    repeats = {}

    for pos, token in enumerate(tensor[0]):
        id  = token.item()

        if id in last_occurrence:
            repeats[pos] = last_occurrence[id]
        last_occurrence[id] = pos
    
    return repeats

def induction_attn_detector(cache: ActivationCache, tokens, off_by_one = True, threshhold = .3) -> List[str]:
    """
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be induction heads

    Remember - the tokens used to generate rep_cache are (bos_token, *rand_tokens, *rand_tokens)
    """
    repeat_dict = find_repeating_rows(t.tensor(tokens))

    def cond(i, j):
        if i not in repeat_dict.keys():
            return False
        to_add = 1 if off_by_one else 0
        return repeat_dict[i] + to_add == j

    return over_threshhold_attn(cache, cond, threshhold=threshhold)
```

</details>

In [ ]:
print("Heads attending to current token  = ", ", ".join(current_attn_detector(gpt_cache)))
print("Heads attending to previous token = ", ", ".join(prev_attn_detector(gpt_cache)))
print("Heads attending to first token    = ", ", ".join(first_attn_detector(gpt_cache)))
print("Heads attending to previous mention = ", ", ".join(induction_attn_detector(gpt_cache, gpt_tokens, off_by_one=False)))
print("Heads attending to one after previous mention = ", ", ".join(induction_attn_detector(gpt_cache, gpt_tokens, off_by_one=True)))


print("Heads attending to current token  = ", ", ".join(current_attn_detector(toymodel_cache)))
print("Heads attending to previous token = ", ", ".join(prev_attn_detector(toymodel_cache)))
print("Heads attending to previous mention = ", ", ".join(induction_attn_detector(toymodel_cache, toymodel_tokens, off_by_one=False)))
print("Heads attending to one after previous mention = ", ", ".join(induction_attn_detector(toymodel_cache, toymodel_tokens, off_by_one=True)))


<details>
<summary>Hint</summary>

Try and compute the average attention probability along the relevant tokens. For instance, you can get the tokens just below the diagonal by using `t.diagonal` with appropriate `offset` parameter:

```python
>>> arr = t.arange(9).reshape(3, 3)
>>> arr
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

>>> arr.diagonal()
tensor([0, 4, 8])

>>> arr.diagonal(-1)
tensor([3, 7])
```

Remember that you should be using `cache["pattern", layer]` to get all the attention probabilities for a given layer, and then indexing on the 0th dimension to get the correct head.
</details>

<details>
<summary>Solution </code></summary>

```python
def current_attn_detector(cache: ActivationCache) -> List[str]:
    '''
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be current-token heads
    '''
    attn_heads = []
    for layer in range(model.cfg.n_layers):
        for head in range(model.cfg.n_heads):
            attention_pattern = cache["pattern", layer][head]
            # take avg of diagonal elements
            score = attention_pattern.diagonal().mean()
            if score > 0.4:
                attn_heads.append(f"{layer}.{head}")
    return attn_heads

def prev_attn_detector(cache: ActivationCache) -> List[str]:
    '''
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be prev-token heads
    '''
    attn_heads = []
    for layer in range(model.cfg.n_layers):
        for head in range(model.cfg.n_heads):
            attention_pattern = cache["pattern", layer][head]
            # take avg of sub-diagonal elements
            score = attention_pattern.diagonal(-1).mean()
            if score > 0.4:
                attn_heads.append(f"{layer}.{head}")
    return attn_heads

def first_attn_detector(cache: ActivationCache) -> List[str]:
    '''
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be first-token heads
    '''
    attn_heads = []
    for layer in range(model.cfg.n_layers):
        for head in range(model.cfg.n_heads):
            attention_pattern = cache["pattern", layer][head]
            # take avg of 0th elements
            score = attention_pattern[:, 0].mean()
            if score > 0.4:
                attn_heads.append(f"{layer}.{head}")
    return attn_heads
```

Note - choosing `0.4` as a threshold is a bit arbitrary, but it seems to work well enough. In this particular case, a threshold of `0.5` results in no head being classified as a current-token head.
</details>


Compare the printouts to your attention visualisations above. Do they seem to make sense?
Play around with the threshhold and see what happens

# Logit Attribution
A consequence of the residual stream is that the output logits are the sum of the contributions of each layer, and thus the sum of the results of each head. This means we can decompose the output logits into a term coming from each head and directly do attribution like this!


Your mission here is to write a function to look at how much each component contributes to the correct logit. Your components are:

* The direct path (i.e. the residual connections from the embedding to unembedding), for each attention head

To emphasise, these are not paths from the start to the end of the model, these are paths from the output of some component directly to the logits - we make no assumptions about how each path was calculated!

A few important notes for this exercise:

* Here we are just looking at the DIRECT effect on the logits, i.e. the thing that this component writes / embeds into the residual stream - if heads compose with other heads and affect logits like that, or inhibit logits for other tokens to boost the correct one we will not pick up on this!
* By looking at just the logits corresponding to the correct token, our data is much lower dimensional because we can ignore all other tokens other than the correct next one (Dealing with a 50K vocab size is a pain!). But this comes at the cost of missing out on more subtle effects, like a head suppressing other plausible logits, to increase the log prob of the correct one.
    * There are other situations where our job might be easier. For instance, in the IOI task (which we'll discuss shortly) we're just comparing the logits of the indirect object to the logits of the direct object, meaning we can use the **difference between these logits**, and ignore all the other logits.


<details>

<summary>Question - why don't we do this to the log probs instead?</summary>

Because log probs aren't linear, they go through `log_softmax`, a non-linear function.
</details>



### Exercise - build logit attribution tool



You should implement the `logit_attribution` function below. This should return the contribution of each component in the "correct direction". 




In [ ]:
import plotly.graph_objects as go

def plot_attribution_pattern(attribution_scores: Float[Tensor, "layers heads"]):
    num_layers, num_heads = attribution_scores.shape
    
    fig = go.Figure(data=go.Heatmap(
        z=attribution_scores,
        x=[f"Head {i+1}" for i in range(num_heads)],
        y=[f"Layer {i+1}" for i in range(num_layers)],
        colorscale='Viridis',
        hoverongaps=False
    ))
    
    fig.update_layout(
        title={
            'text': "Attribution Scores",
            'font': {'size': 16}
        },
        xaxis_title="Heads",
        yaxis_title="Layers",
        xaxis={'tickfont': {'size': 12}, 'tickangle': -45},
        yaxis={'tickfont': {'size': 12}},
        coloraxis_colorbar={
            'title': '',
            'tickfont': {'size': 12}
        }
    )
    
    fig.show()

In [ ]:
def logit_attribution(
    tokens: Int[Tensor, "batch seq"],
    model: HookedTransformer,
    cache: ActivationCache,
    token_position: int
) -> Float[Tensor, "layers heads"]:
    """
    Computes the logit attribution for a specific token position in the input sequence.

    Args:
        tokens (Int[Tensor, "batch seq"]): The input token IDs tensor with shape (batch_size, sequence_length).
        model (HookedTransformer): The HookedTransformer model instance.
        cache (ActivationCache): The activation cache containing the intermediate results.
        token_position (int): The position of the token in the input sequence for which to compute the attribution.

    Returns:
        Float[Tensor, "layers heads"]: The logit attribution tensor with shape (num_layers, num_heads).

    Description:
        This function computes the logit attribution for a specific token position in the input sequence.
        It unembeds the output of each attention head in each layer, and sees what upweight it gives on the correct next token.

    Note:
        - The input `tokens` tensor is assumed to have a batch size of 1.
        - The `token_position` is zero-indexed, meaning the first token in the sequence has a position of 0.
        - The returned attention pattern has shape (num_layers, num_heads), representing the attribution scores
          for each layer and attention head.
    """
    # Retrieve the attention results from the activation cache for each transformer block

    # Hide: all
    results = ________
    # Hide: none

    # Stack the attention results along the layer dimension

    # Hide: all
    results = t.stack________
    # Hide: none
    
    # Select the attention results corresponding to the specified token position

  
    # Pass the selected attention results through the model's unembed function to obtain the logits

    # Hide: all
    logits = ________
    # Hide: none
    
    # Get the ID of the next token in the sequence

    # Hide: all
    next_token_id = ________
    # Hide: none
    
    # Extract the logits corresponding to the next token ID

    # Hide: all
    attributions = ________
    # Hide: none
    
    return attributions
test_logit_attribution(logit_attribution, toy_transformer)

<details>

<summary>Solution</summary>


```python
def logit_attribution(
    tokens: Int[Tensor, "batch seq"],
    model: HookedTransformer,
    cache: ActivationCache,
    token_position: int
) -> Float[Tensor, "layers heads"]:
    """
    Computes the logit attribution for a specific token position in the input sequence.

    Args:
        tokens (Int[Tensor, "batch seq"]): The input token IDs tensor with shape (batch_size, sequence_length).
        model (HookedTransformer): The HookedTransformer model instance.
        cache (ActivationCache): The activation cache containing the intermediate results.
        token_position (int): The position of the token in the input sequence for which to compute the attribution.

    Returns:
        Float[Tensor, "layers heads"]: The logit attribution tensor with shape (num_layers, num_heads).

    Description:
        This function computes the logit attribution for a specific token position in the input sequence.
        It unembeds the output of each attention head in each layer, and sees what upweight it gives on the correct next token.

    Note:
        - The input `tokens` tensor is assumed to have a batch size of 1.
        - The `token_position` is zero-indexed, meaning the first token in the sequence has a position of 0.
        - The returned attention pattern has shape (num_layers, num_heads), representing the attribution scores
          for each layer and attention head.
    """
    # Retrieve the attention results from the activation cache for each transformer block

    # Hide: all
    results = [cache[f"blocks.{i}.attn.hook_result"] for i in range(len(model.blocks))]
    # Hide: none

    # Stack the attention results along the layer dimension

    # Hide: all
    results = t.stack(results, dim=1)
    # Hide: none
    
    # Select the attention results corresponding to the specified token position

    # Hide: all
    results = results[token_position, :, :, :]
    # Hide: none
    
    # Pass the selected attention results through the model's unembed function to obtain the logits

    # Hide: all
    logits = model.unembed(results)
    # Hide: none
    
    # Get the ID of the next token in the sequence

    # Hide: all
    next_token_id = tokens[0, token_position + 1]
    # Hide: none
    
    # Extract the logits corresponding to the next token ID

    # Hide: all
    attributions = logits[:, :, next_token_id]
    # Hide: none
    
    return attributions
```

</details>

Now, that you have to tool, to see wich heads have wich effects on the logits, run some experiments to see, wich heads are usefull for induction.
Are they the same, that have the pattern you identified earlier?

In [ ]:
rand_seq = t.randint(1000, 10000, (1, 10))
repeat_seq = t.cat([rand_seq, rand_seq], dim=1)

# Hide: hard
gpt_logits, gpt_cache = gpt2_small.run_with_cache(repeat_seq, remove_batch_dim=True)
toymodel_logits, toymodel_cache = toy_transformer.run_with_cache(repeat_seq, remove_batch_dim=True)

token_postion = 14
attribution_scores = logit_attribution(repeat_seq, toy_transformer, toymodel_cache, token_postion)
plot_attribution_pattern(attribution_scores)
# Hide: none